# Ali Kamal
# i191865@nu.edu.pk

In [1]:
import numpy as np
import re
import difflib
import string
import itertools
from collections import Counter
from operator import itemgetter

# Utility Functions

In [2]:
"""
Gets a text file as input and returns a 
stripped list containing the text
"""
def read(f):
    with open(f) as file:
        return [(line.rstrip()) for line in file.readlines() if line.strip()]


"""
Gets a list of text as input, and returns
ngram (bigram,trigram,4-gram etc) depending
on input parameters
"""
#1. Train a uni-gram model using the corpus provided in data.txt.
def get_ngrams(text,ngram=1):
    words=[word for word in text.split(" ")]
    return [' '.join(ngram) for ngram in zip(*[words[i:] for i in range(0,ngram)])]


"""
Gets a list of ngrams and returns 
a dictionary with ngrams as keys
and their counts in the corpus as values
"""
def get_ngram_counts(ngrams):
    return Counter(np.hstack(ngrams))


"""
Increments edit table dictionary, or 
makes a new entry on the dictionary
if they key does not exist
"""
def increment_dic(dic,correct,wrong):
    try:
        dic[correct+","+wrong]+=1
    except KeyError:
        dic[correct+","+wrong]=1


"""
Takes misspelings text file as input and
generates respective edit tables based on 
the manner of misspellings (insert,delete,substitute,transpose)
"""
#2. Create insert, delete, substitution and transposition tables for alphabets a-z
#   using the provided misspellings.txt. The tables can be implemented using Python dictionaries
def generate_edit_tables(data,insert=dict(),delete=dict(),substitute=dict(),transpose=dict()):
    with open(data) as file:
        next(file)
        for line in file.readlines():
            separated_line=line.split()
            correct_word=re.sub(",","",separated_line[0])
            wrong_words=separated_line[1:]
            for idx,wrong_word in enumerate(wrong_words):
                if(len(wrong_word)!=len(correct_word)): #Insert,Delete Error
                    if(len([elem for elem in list(difflib.ndiff(correct_word,wrong_word))if('+'in elem or'-'in elem)])==1):
                        for idx2,i in enumerate(difflib.ndiff(correct_word, wrong_word)):
                            if i[0]==' ': #No difference
                                continue
                            elif i[0]=='+': #Insert Error
                                #X=>XY (X=Correct,Y=Incorrect), dictionary like this: dic[X,Y]
                                increment_dic(insert,'@',i[-1]) if idx2==0 else increment_dic(insert,correct_word[idx2-1],i[-1])
                                #@ represents start of string
                            elif i[0]=='-': #Delete Error
                                #XY=>X (X=Correct,Y=Correct), dictionary like this: dic[X,Y]
                                increment_dic(delete,'@',i[-1]) if idx2==0 else increment_dic(delete,correct_word[idx2-1],i[-1])
                else: #Substitute,Transpose Error
                    if(len([elem for elem in list(difflib.ndiff(correct_word,wrong_word))if('+'in elem or'-'in elem)])==2):
                        for idx,i in enumerate(wrong_word):
                            if(i!=correct_word[idx]):       
                                if(idx!=len(wrong_word)-1 and wrong_word[idx+1]==correct_word[idx]): #Transpose Error                           
                                    #XY=>YX (XY=Correct,YX=Incorrect), dictionary like this: dic[X,Y]
                                    increment_dic(transpose,correct_word[idx],i)
                                else:
                                    #X=>Y (X=Correct,Y=Incorrect), dictionary like this: dic[Y,X]
                                    increment_dic(substitute,i,correct_word[idx])                 
    return dict(sorted(insert.items())),dict(sorted(delete.items())),dict(sorted(substitute.items())),dict(sorted(transpose.items()))


"""
Returns error model probabilites P(x|w) with 
laplace smoothing, corresponding to the 
respective error model table
"""
#3. Create a function that calculates P(x|w) using the Error Model tables.
def prob_error_model(wrong_word,idx,error_model_table,vocab_len,op,correct_word=None):
    #Calculating probabilities WITH laplace smoothing
    try:
        if(idx==0 and op=='insert'):
            return (error_model_table['@'+','+wrong_word[idx]]+1)/(sum(error_model_table.values())+vocab_len)
        elif(idx==0 and op=='delete'):
            return (error_model_table['@'+','+correct_word]+1)/(sum(error_model_table.values())+vocab_len)
        elif(idx>0 and op=='insert'):
            return (error_model_table[wrong_word[idx-1]+','+wrong_word[idx]]+1)/(sum(error_model_table.values())+vocab_len)
        elif(idx>0 and op=='delete'):
            return (error_model_table[wrong_word[idx-1]+','+correct_word]+1)/(sum(error_model_table.values())+vocab_len)
        elif(op=='substitute'):
            return (error_model_table[wrong_word[idx]+','+correct_word]+1)/(sum(error_model_table.values())+vocab_len)
        elif(op=='transpose'):
            return (error_model_table[wrong_word[idx+1]+','+wrong_word[idx]]+1)/(sum(error_model_table.values())+vocab_len)
    except KeyError:
        return (1)/(sum(error_model_table.values())+vocab_len)


"""
Takes an incorrect word and generates
correct candidate words for it, provided they are
within 1 edit distance(insert,delete,transpose,substitute).
"""
#4. Create a function that returns the set of candidate correct words for a given word x. The
#   set of candidatewords must be a subset of the vocabulary V . This means that for each
#   candiate word w, P(w) > 0.
def generate_candidate_words(wrong_word,unigrams,insert,delete,substitute,transpose,k):
    vocab_len=len(unigrams.keys())
    len_word=len(wrong_word)
    ins=dict()
    dl=dict()
    sub=dict()
    trans=dict()
    for idx,i in enumerate(wrong_word):
        modified_word_ins=wrong_word[:idx]+''+ wrong_word[idx + 1:]
        if(idx!=len_word-1 and len_word>1):
            modified_word_trans=''.join((wrong_word[:idx],wrong_word[idx+1],wrong_word[idx+1:idx+1],wrong_word[idx],wrong_word[idx+2:]))
            #transpose (if swapping of two adjacent characters from wrong_word results in candidate word present in Vocabulary)
            if(modified_word_trans in unigrams.keys()):
                trans[modified_word_trans]=prob_error_model(wrong_word,idx,transpose,vocab_len,'transpose')
        #insert (if deletion of word from wrong_word results in candidate word present in Vocabulary)
        if(modified_word_ins in unigrams.keys()):
            ins[modified_word_ins]=prob_error_model(wrong_word,idx,insert,vocab_len,'insert')
        for c in list(map(chr,range(ord('a'),ord('z')+1))):
            if(c!=i):
                modified_word_sub=wrong_word[:idx] + c + wrong_word[idx + 1:]
                #substitute (if substitution of word in wrong_word results in candidate word resent in Vocabulary)
                if(modified_word_sub in unigrams.keys()):
                    sub[modified_word_sub]=prob_error_model(wrong_word,idx,substitute,vocab_len,'substitute',c)
            modified_word_dl=wrong_word[:idx] + c + wrong_word[idx:]
            #delete (if insertion of word in wrong_word results in candidate word resent in Vocabulary)
            if(modified_word_dl in unigrams.keys()):
                dl[modified_word_dl]=prob_error_model(wrong_word,idx,delete,vocab_len,'delete',c)
    #returns top k candidate words of each error type having the most probability
    return dict(sorted(ins.items(),key=itemgetter(1),reverse=True)[:k]),dict(sorted(dl.items(),key=itemgetter(1),reverse=True)[:k]),dict(sorted(sub.items(),key=itemgetter(1),reverse=True)[:k]),dict(sorted(trans.items(),key=itemgetter(1),reverse=True)[:k])


"""
Takes a sentence as input and and generates all combinations
of candidate words and the original sentence
"""
#5. Create a function that returns a list of candidate correct sentences 
#   created using all combinations of your candidate correct words and the original sentence.
def generate_candidate_sentences(sentence,unigrams,insert,delete,substitute,transpose,k):
    sentence=sentence.translate(str.maketrans('', '', string.punctuation))
    sentence = sentence.split() if isinstance(sentence, str) else sentence
    candidate_sentences=dict()
    probs=dict()
    for s in sentence:
        ins,dl,sub,trans=generate_candidate_words(s,unigrams,insert,delete,substitute,transpose,k)
        candidate_sentences.setdefault(s,[]).append(s)#8. Your solution must also be able to handle cases
                                                      #   when there are no candidate words.
        [candidate_sentences.setdefault(s,[]).append(key) for key in ins.keys()]
        [candidate_sentences.setdefault(s,[]).append(key) for key in dl.keys()]
        [candidate_sentences.setdefault(s,[]).append(key) for key in sub.keys()]
        [candidate_sentences.setdefault(s,[]).append(key) for key in trans.keys()]
        probs.update(ins)
        probs.update(dl)
        probs.update(sub)
        probs.update(trans)
    #Gets all combinations of candidate correct words, with a limit of 1 MILLION combinations max, in order to improve 
    #compute performance. Else, we quickly run out of RAM
    candidates=itertools.islice(itertools.product(*(candidate_sentences[key] for key in candidate_sentences)),10000000)
    return list(candidates),probs


"""
Returns probability of candidate sentence 
w.r.t error model tables
"""
def get_error_model_probs(candidates,error_model_probs):
    probabilities=[]
    for sen in candidates:
        prob=0
        for word in sen:
            try:
                prob+=error_model_probs[word]
            except:
                pass
        probabilities.append(prob)
    return probabilities


"""
Returns probability of candidate sentence 
w.r.t bigram model
"""
def get_bigram_probs(candidates,bigrams,error_model_probs,vocab_len):
    probabilities=[]
    laplace_denom=sum(bigrams.values())+vocab_len
    bigrams_prob=dict()
    for sen in candidates:
        prob=0
        for idx,word in enumerate(sen):
            try:
                if(not idx):#First Word
                    prob+=error_model_probs[word]
                else:
                    try:
                        prob+=bigrams_prob[sen[idx-1]+" "+word]
                    except KeyError:
                        bigrams_prob[sen[idx-1]+" "+word]=(bigrams[sen[idx-1]+" "+word]+1)/laplace_denom
                        prob+=bigrams_prob[sen[idx-1]+" "+word]
            except:
                pass
        probabilities.append(prob)
    return probabilities


"""
Returns probabilities of candidate sentences generated
through bigram model and error model table
"""
#6. Train an ensemble of atleast one character-level language model and atleast
#   one word-level language model using the corpus provided in data.txt.
def ensemble_model(candidates,error_model_probs,unigrams,bigrams):
    vocab_len=len(unigrams.keys())
    #Getting probabilities of candidate sentences w.r.t edit tables
    em=get_error_model_probs(candidates,error_model_probs)
    #Getting probabilities of candidate sentences w.r.t bigram
    bp=get_bigram_probs(candidates,bigrams,error_model_probs,vocab_len)
    return np.array(bp),np.array(em)

# Main Generator Function

In [3]:
"""
Returns candidate sentence which has highest final probability,
after getting probabilities through ensemble model
"""
#7. Create a function that takes a list of candidate correct sentences as input
#   and returns the candidate correct sentence with the highest probability
#   according to your ensemble of language models.
def get_best_candidate_sentence(candidates,error_model_probs,unigrams,bigrams):
    bp,em=ensemble_model(candidates,error_model_probs,unigrams,bigrams)
    #Adding both probabilities, one from bigrams and the other from error model tables, with some 
    #parameters to reduce impact of error model probabilities, giving more weight to bigram probabilities,
    #but not completely disregarding error modle probabilities
    final_prob=bp + (em/4)
    #returning argmax of highest probabilities
    return " ".join(candidates[np.argmax(final_prob)])

<br><br>
# Driver Code

In [4]:
#Generating unigrams and bigrams
data=read("data.txt")
unigrams=[]
bigrams=[]   
for i in data:
    unigrams.append(get_ngrams(i))
    bigrams.append(get_ngrams(i,2))
unigrams=get_ngram_counts(unigrams)
bigrams=get_ngram_counts(bigrams)
#Deleting bigram keys which have count of only 1, to reduce memory footprint.
#Keys with counts of 1 are not that important to us, plus we are already using laplace smoothing,
#so even if we do need a key(in the future) that we are deleting, this should be handled by laplace smoothing.
[bigrams.pop(k) for k,v in list(bigrams.items()) if v==1];

#### Generating error model edit tables

In [5]:
#Generating respective Error Model Edit Tables
insert,delete,substitute,transpose=generate_edit_tables("misspellings.txt")

#### Generating candidate sentences from candidate words for sample incorrect sentence, and getting best candidate sentence according to our ensemble model

In [ ]:
#Generating all combinations of candidate words from original sentence

incorrect_sentence='mvin pcakistan mevn rwhta houqn eor scahool wfaloun xke savh NLP parhnla hchahta hounp'

#Gets top k candidate words according to error model probability. Getting top k in order to improve 
#compute performance, else we quickly run out of RAM
k=5
candidates,error_model_probs=generate_candidate_sentences(incorrect_sentence,unigrams,insert,delete,substitute,transpose,k)

#Gets best candidate sentence according to our ensemble model
correct_sentence=get_best_candidate_sentence(candidates,error_model_probs,unigrams,bigrams)

#### Printing our chosen best candidate sentence

In [ ]:
print(correct_sentence)